In [1]:
jfk = (40.64417, -73.78222)
lga = (40.77472, -73.87194)
ewr = (40.68972, -74.175)

In [2]:
import pandas as pd
data_hail = pd.read_csv("hailForStats.csv",sep='\t')

In [6]:
data_hail['lpep_pickup_datetime'].max()

'2016-12-31 23:59:59'

In [7]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6367 * 1000 # Radius of earth in meters. Use 3956 for miles
    return c * r

In [8]:
data_hail['pickup_distance_to_jfk']=data_hail.apply(lambda row:haversine(jfk[1], jfk[0], row['Pickup_longitude'], row['Pickup_latitude']),axis=1)

In [9]:
data_hail['pickup_distance_to_lga']=data_hail.apply(lambda row:haversine(lga[1], lga[0], row['Pickup_longitude'], row['Pickup_latitude']),axis=1)

In [10]:
data_hail['pickup_distance_to_ewr']=data_hail.apply(lambda row:haversine(ewr[1], ewr[0], row['Pickup_longitude'], row['Pickup_latitude']),axis=1)

In [ ]:
air_data = data_hail[(data_hail['pickup_distance_to_jfk']<1000)|(data_hail['pickup_distance_to_lga']<1000)|(data_hail['pickup_distance_to_ewr']<1000)]

In [ ]:
len(air_data)

In [ ]:
# air_data['Lpep_dropoff_datetime'].iloc[0]-air_data['lpep_pickup_datetime'].iloc[0]
air_data['Lpep_dropoff_datetime'] = pd.to_datetime(air_data['Lpep_dropoff_datetime'],errors="coerce")
air_data['lpep_pickup_datetime'] = pd.to_datetime(air_data['lpep_pickup_datetime'],errors="coerce")

In [ ]:
def get_time_in_sec(time1, time2):
    timedelta = time1-time2
    return timedelta.days * 24 * 3600 + timedelta.seconds

air_data['trip_time_in_secs'] = air_data.apply(lambda row: get_time_in_sec(row['Lpep_dropoff_datetime'], row['lpep_pickup_datetime']), axis=1)

In [ ]:
air_data.to_csv("air_data.csv",sep='\t')